<h1><center>PISAP: Python Interactive Sparse Astronomical Data Analysis Packages</center></h1>
<h2><center>Anstronomic/Neuroimaging common structure overview</center></h2>
<div style="text-align: center">Credit: </div>

Pisap is a Python package related to sparsity and its application in
astronomical or mediacal data analysis. This package also bind the 'sparse2d' C++ library
that allows fast and extended sparse decomposition, denoising and deconvolution.
It is accessible to everybody, and is reusable in various contexts.
The project is hosted on github: https://github.com/neurospin/pisap.<br><br>

<h3>First check</h3>

In order to test if the 'pisap' package is installed on your machine, you can check the package version:

In [1]:
import pisap

print pisap.__version__

/home/hc252523/.local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


0.0.0


<h2>Import astronomical data</h2>

The package provides a common interface to import and visualize astronomical FITS dataset. It also embeds a set of toy dataset that will be used during this tutorial:



In [2]:
import pisap
from pprint import pprint
from pisap.data import get_sample_data

fits_data_path = get_sample_data("astro-fits")
image = pisap.io.load(fits_data_path)
print image.shape, image.spacing, image.data_type
pprint(image.metadata)
print image.data.dtype
image.show()

(128, 128) [ 1.  1.] scalar
{'BITPIX': -64,
 'COMMENT': "  and Astrophysics', volume 376, page 359; bibcode: 2001A&A...376..359H",
 'EXTEND': True,
 'NAXIS': 2,
 'NAXIS1': 128,
 'NAXIS2': 128,
 'SIMPLE': True,
 'path': '/home/hc252523/.local/share/pisap/M31_128.fits'}
float32


<h2>Import neuroimaging data</h2>

The package provides a common interface to import and visualize neuroimaging NIFTI dataset. It also embeds a set of toy dataset that will be used during this tutorial:



In [3]:
import pisap
from pprint import pprint
from pisap.data import get_sample_data

nifti_data_path = get_sample_data("mri-nifti")
image = pisap.io.load(nifti_data_path)
image.scroll_axis = 2
print image.shape, image.spacing, image.data_type
pprint(image.metadata)
print image.data.dtype
image.show()

(240, 256, 160) [ 1.          1.          1.10000002] scalar
{'path': '/home/hc252523/.local/share/pisap/t1_localizer.nii.gz'}
float32


<h2>Decompose/recompose an image using a fast ISAP/C++ based discrete wavelet transform</h2>

The package provides also a common interface to the ISAP C++ software developped by the COSMOSTAT lab. The code is optimzed and give access to many decompsition strategies. All the ISAP library decompositions have been declared in a registery:

In [4]:
from pprint import pprint
import pisap.extensions.transform
from pisap.base.transform import WaveletTransformBase
pprint(WaveletTransformBase.REGISTRY)

{'BsplineWaveletTransformATrousAlgorithm': <class 'pisap.extensions.transform.BsplineWaveletTransformATrousAlgorithm'>,
 'DecompositionOnScalingFunction': <class 'pisap.extensions.transform.DecompositionOnScalingFunction'>,
 'FeauveauWaveletTransformWithoutUndersampling': <class 'pisap.extensions.transform.FeauveauWaveletTransformWithoutUndersampling'>,
 'HaarWaveletTransform': <class 'pisap.extensions.transform.HaarWaveletTransform'>,
 'HalfPyramidalTransform': <class 'pisap.extensions.transform.HalfPyramidalTransform'>,
 'IsotropicAndCompactSupportWaveletInFourierSpace': <class 'pisap.extensions.transform.IsotropicAndCompactSupportWaveletInFourierSpace'>,
 'LinearWaveletTransformATrousAlgorithm': <class 'pisap.extensions.transform.LinearWaveletTransformATrousAlgorithm'>,
 'MallatWaveletTransform79Filters': <class 'pisap.extensions.transform.MallatWaveletTransform79Filters'>,
 'MeyerWaveletsCompactInFourierSpace': <class 'pisap.extensions.transform.MeyerWaveletsCompactInFourierSpace'>

We illustrate the the decompose/recompose using a 79 wavelet transform:

In [5]:
import pisap.extensions.transform
from pisap.base.transform import WaveletTransformBase
from pisap.data import get_sample_data

nifti_data_path = get_sample_data("mri-slice-nifti")
image = pisap.io.load(nifti_data_path)
transform = WaveletTransformBase.REGISTRY["MallatWaveletTransform79Filters"](nb_scale=4, verbose=1)
transform.data = image
transform.analysis()
transform.show()
rec_image = transform.synthesis()
rec_image.show()

[info] Executing ISAP command: mr_transform -t 14 -n 4 -U 2 -v -T 1 -L /tmp/tmpRh2hit/in.fits /tmp/tmpRh2hit/cube.mr...
[info] Executing ISAP command: mr_recons -v /tmp/tmp9BjChG/cube.mr /tmp/tmp9BjChG/out.fits...
